# NEXT WORD PREDICTION

## By Ananya Kumar Mamillapalli

### Importing Libraries

In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
import pickle

In [2]:
file= open('1661-0.txt',encoding='UTF-8').read().lower()

In [3]:
print('corpus length:\t', len(file))

corpus length:	 581888


In [4]:
tokinizer=RegexpTokenizer(r'\w+')
pickle.dump(tokinizer,open('token.pkl','wb'))

In [5]:
words= tokinizer.tokenize(file)

In [6]:
len (words)

109226

In [7]:
unique_words= np.unique(words)
len(unique_words)

8201

In [8]:
unique_word_index= dict((c,i) for i,c in enumerate(unique_words))

### Preparing Data

In [9]:
word_length=4
prev_words=[]
next_words=[]
for i in range(len(words)-word_length):
    prev_words.append(words[i:i + word_length])
    next_words.append(words[i + word_length])

print('pev wods:\t', prev_words[1])
print('next wods:\t', next_words[1])

pev wods:	 ['gutenberg', 's', 'the', 'adventures']
next wods:	 of


In [10]:
X = np.zeros((len(prev_words), word_length, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [11]:
print(X[0][0])

[False False False ... False False False]


### Model Building

In [12]:
model = Sequential()
model.add(LSTM(128, input_shape=(word_length, len(unique_words))))
model.add(Dense(len(unique_words),activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               4264960   
_________________________________________________________________
dense (Dense)                (None, 8201)              1057929   
Total params: 5,322,889
Trainable params: 5,322,889
Non-trainable params: 0
_________________________________________________________________


In [13]:
from tensorflow.keras.optimizers import RMSprop

In [14]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/20
811/811 [==============================] - 212s 259ms/step - loss: 6.0062 - accuracy: 0.1074 - val_loss: 7.0047 - val_accuracy: 0.1014
Epoch 2/20
811/811 [==============================] - 207s 256ms/step - loss: 5.7673 - accuracy: 0.1473 - val_loss: 8.0781 - val_accuracy: 0.0998
Epoch 3/20
811/811 [==============================] - 205s 253ms/step - loss: 5.7446 - accuracy: 0.1743 - val_loss: 8.1996 - val_accuracy: 0.1108
Epoch 4/20
811/811 [==============================] - 206s 254ms/step - loss: 5.4738 - accuracy: 0.2096 - val_loss: 8.3132 - val_accuracy: 0.1100
Epoch 5/20
811/811 [==============================] - 207s 256ms/step - loss: 5.2256 - accuracy: 0.2468 - val_loss: 8.2927 - val_accuracy: 0.0947
Epoch 6/20
811/811 [==============================] - 209s 257ms/step - loss: 5.0046 - accuracy: 0.2827 - val_loss: 8.3941 - val_accuracy: 0.0923
Epoch 7/20
811/811 [==============================] - 206s 254ms/step - loss: 4.8170 - accuracy: 0.3209 - val_loss: 8.5665 -

In [15]:
model.save('next_wod.h5')

### Testing on user input data

In [16]:
def prep_input(txt):
    x=np.zeros((1, word_length, len(unique_words)))
    for t, word in enumerate(txt.split()[-4:]):
        #print(word)
        x[0, t, unique_word_index[word]] = 1
    index=np.argmax(model.predict(x))
    rslt=unique_words[index]
    return rslt

In [17]:
unique_words[2001]

'delight'

In [18]:
txt = input('Enter text :')

prep_input(txt.lower())

Enter text :I am watching


'hands'